### Processador conceitual em Python
### Sistemas de Computação
### Jonathas dos Santos Andrade

### Introdução
Para iniciar o desenvolvimento do processador conceitual em Python, é preciso indicar as variáveis do tipo lista que irão conter os valores dos dados da memória principal e do cache. Além disso, de forma contida no escopo global do código, as variáveis referentes ao contador de instrução e ao registrador também serão definidas.

In [2]:
from random import randint

MP = []
CACHE = []
REG = ""
CI = 0

Da mesma forma, é importada a biblioteca random para a inserção dos valores de forma aleatória dentro do cache durante a manipulação dos dados no seu controlador.

### O iniciador da memória principal

Em continuidade, é preciso definir uma função iniciadora da memória principal que indique a quantidade de células e tamanho de cada uma delas. Nessa lógica, faz-se necessário o uso de funções de repetição do python para criar esses valores. No mais, como está sendo utilizado listas para representar esses componentes, as células serão listas dentro das listas, assumindo um formato matricial. 

In [3]:
def mpInitiator(cells: int, size: int):
    # Inicializa a memória principal com a quantidade de células definida como parâmetro
    for x in range(cells):
        cell = []
        for y in range(size):
            cell.append(0)
        MP.append(cell)

### O iniciador do cache

De forma semelhante ao método anterior, também é definida uma função de formação do cache utilizando os mesmos métodos de matrizes dentro de listas.

In [4]:
def cacheInitiator(cells: int, size: int):
    # Inicializa a memória cache com a quantidade de células definida como parâmetro
    for l in range(cells):
        cell = []
        for y in range(size):
            cell.append(0)
        CACHE.append(cell)

### O controlador da memória principal

Nesse ponto, para a manipulação das informações que são passadas a memória principal, é definido a função referente ao controlador dessa memória, o qual permite a leitura ou gravação de dados referentes ao endereço que é requerido como parâmetro dessa função.

In [5]:
def mpController(action: bool, addres: str, data: str):
    decimalAddress = numberConversor(addres, 'bin->dec', 8)

    # true = Gravação / false = Leitura
    if(action):
        if len(data) < len(MP[0]):
            tmpData = list(data)
            tmpData.reverse()
            while len(tmpData) < len(MP[0]):
                tmpData.append(0)
            MP[decimalAddress] = tmpData[::-1]
        elif len(data) > len(MP[0]):
            return print(f"Error: o tamanho da palavra não pode exceder o tamanho de {len(MP[0])} bits.")
        else:
            MP[decimalAddress] = list(data)
            return print(f"Success: o dado foi armazenado no index {decimalAddress} da memória.")
    else:
        return ''.join(str(i) for i in MP[decimalAddress])

Para resumir seu funcionamento, pode-se observar que o valor do endereço é recebido em binário e dentro da função é convertido para decimal, de maneira a facilitar a utilização do endereço durante a criação do código (melhor leitura pelo programador do código e pelo leitor). Após isso, é verificado se o tipo da ação passada ao controlador é de leitura ou gravação de dados. Caso for o primeiro, é buscada dentro da memória a célula (lista) referente ao index (endereço em decimal) indicado e, após isso, a substituição dos dados. Por outro lado, se for o segundo, é somente retornado a lista convetida para texto referente ao endereço passado. 

### O controlador da cache

Diferentemente do controlador da MP, a função de controle das informações que são requeridas a cache recebe apenas o endereço em binário da informação na memória principal. Com posse desse dado, o controlador fragmenta o endereço em campos necessários para a localização espacial da informação em campos refentes ao byte, ao conjunto e ao tag que é representado pelo endereço do banco armazenado na linha dentro do conjunto indicado. Por conseguinte, como a estrutura bruta do cache é similar a memória principal, ou seja, em linhas e células, é necessário separar os conjuntos (dado que o tipo de mapeamento utiliado é o de associação por conjuntos) em listas dentro de uma variável para facilitar a busca nas funções de repetição. Assim, depois de converter o campo do conjunto para decimal, é relacionado em uma nova variável o grupo respectivo ao index mencionado, sendo possível notá-la através da variável _currentGroup_.

In [6]:
def cacheController(address: str):
    byteField = numberConversor(address[6::], 'bin->dec', 8)
    groupField = numberConversor(address[4:6], 'bin->dec', 8)
    tagField = numberConversor(address[0:4], 'bin->dec', 8)

    groups = [CACHE[0:4], CACHE[4:8], CACHE[8:12], CACHE[12:16]]
    currentGroup = groups[groupField]

    for x in range(len(currentGroup)):
        dataToString = ''.join(str(i) for i in currentGroup[x])
        currentTagField = numberConversor(dataToString[0:4], 'bin->dec', 8)
        bytesInLine = [dataToString[40::], dataToString[28:40],
                       dataToString[16:28], dataToString[4:16]]

        if int(tagField) == int(currentTagField):
            # verifica se o dado é realmente o que o usuário quer fazendo um pulso a mp
            dataInMp = ''.join(str(i)
                               for i in MP[numberConversor(address, 'bin->dec', 8)])

            if bytesInLine[byteField] == dataInMp:
                return bytesInLine[byteField]

    # se chegar nesse ponto, significa que não achou (miss)
    newCell = []
    tmpCell = []
    conjAddress = 0

    for l in range(len(MP)):
        currentAddress = str(numberConversor(l, 'dec->bin', 9))
        currentAddress = currentAddress[1::]
        blockAddress = currentAddress[0:6]

        if address[0:6] == blockAddress:
            conjAddress = int(numberConversor(
                "".join(str(e) for e in blockAddress[4:6]), "bin->dec", 8))

            for x in MP[l]:
                newCell.append(x)

    newCell = [list(address[0:4]), newCell[36:48], newCell[24:36],
               newCell[12:24], newCell[0:12]]

    for l in range(len(newCell)):
        for x in newCell[l]:
            tmpCell.append(x)

    newCell = tmpCell

    if len(newCell) == 52:
        randomValue = randint(conjAddress-1, conjAddress+1)
        CACHE[randomValue] = newCell

        bytesInLine = [newCell[40::], newCell[28:40],
                       newCell[16:28], newCell[4:16]]

        return ''.join(str(x) for x in bytesInLine[byteField])

    else:
        return print("Error")

Depois que as associações e denominações são realizadas, é feito uma busca dentro do grupo indicado no endereço pela linha que contém o mesmo tag também passado através do endereço, de forma a validar que aquela linha contém o byte buscado. Se o tag for validado, é feito outra verificação para certificar que a informação armazenada na cache é realmente a que está sendo buscada. Caso essa condição também seja validada, o byte é então retornado. Em contrapartida, se nenhuma dessas etapas estiverem de acordo, significa que a informação buscada não está alojada no cache. Dessa forma, é feito então a busca do banco dentro da memória principal que contém aquela informação para ser transmitida ao cache e posteriormente retornada ao chamador da função. No mais, se nenhum desses passos forem realizados, é indicado um erro interno pelo controlador.

### O conversor de valores

Em outro ponto, para facilitar a busca de instruções na memória e entendimento do código por quem está desenvolvendo, foi criado uma função que converte os valores, estejam eles em binário ou decimal, para a unidade que é passada como parâmetro. Caso a conversão requerida seja de decimal para binário, é solicitado pela função o tamanho em bits do resultado.

In [7]:
def numberConversor(value, type, size):
    convertedValue = 0

    if type == "dec->bin":
        # efetua a conversão do valor em decimal para binário
        number = int(value)
        string = ''

        while number > 0:
            rest = int(number % 2)
            string += str(rest)
            number = (number - rest) / 2

        if(len(string) < size):
            tmpList = list(string)

            while len(tmpList) < size:
                tmpList.append('0')

            convertedValue = ''.join(tmpList[::-1])

    if type == "bin->dec":
        # faz a conversão do valor em binário para decimal
        result = 0
        stringToList = list(value)

        for i in range(len(stringToList)):
            currentValue = int(stringToList[i])
            if(currentValue != 1 and currentValue != 0):
                return print("Por favor, o valor deve estar em binário! Tente novamente.")
            else:
                result += currentValue * \
                    2 ** (len(stringToList) - i - 1)

        convertedValue = int(result)

    return convertedValue


### O iniciador do programa

Por fim, para a conclusão do processador conceitual, é preciso inicializar as memórias e criar uma repetição contínua de forma a manter a execução do código. Após isso, são demonstradas as opções do programa para o cliente, e caso a primeira opção seja selecionada, as instruções são previamente escritas na memória principal e a sua leitura e execução é iniciada e mantida enquanto o contador de instrução estiver de acordo com a condição passada na função de repetição. No mais, a associação entre o valor e sua referida instrução é realizada através de funções condicionais e dentro delas é exposta ao cliente o estado dos registradores e contadores durante a sua execução.

In [8]:
# inicializa a memória principal definindo o tamanho de cada célula e a quantidade de células
mpInitiator(256, 12)
# inicializa a memória cache definindo o tamanho de cada célula e a quantidade de células
# nesse caso, como a mp será composta por blocos de 4 bytes, cada linha terá 4 bytes de tamanho + 4 bits pro tag
# e um total de 16 linhas / 4 linhas = 4 conjuntos -> - 4 bit pro tag - 2 bits pro conjunto - 2 bits pro campo byte
cacheInitiator(16, 52)

# mantém a execução do programa enquanto o runner estiver em nível alto
runner = True

while runner:
    print("""
    Selecione uma opção:
    1 - Carregar programa
    2 - Visualizar MP
    3 - Visualizar cache
    4 - Encerrar
    """)
    cmd = int(input(""))

    # o comando 1 inicializa o programa proposto na atividade
    if cmd == 1:
        # grava a instrução 1b4 no endereço 00
        mpController(True, '00000000', '000110110100')
        # grava a instrução 4b5 no endereço 01
        mpController(True, '00000001', '010010110101')
        # grava a instrução 705 no endereço 02
        mpController(True, '00000010', '011100000101')
        # grava a instrução ab4 no endereço 03
        mpController(True, '00000011', '101010110100')
        # grava a instrução 806 no endereço 04
        mpController(True, '00000100', '100000000110')
        # grava a instrução ab5 no endereço 05
        mpController(True, '00000101', '101010110101')
        # grava a instrução 000 (end) no endereço 06
        mpController(True, '00000110', '000000000000')
        # grava o valor 2b3 na posíção b4
        mpController(True, '10110100', '001010110011')
        # grava o valor 1a3 na posíção b5
        mpController(True, '10110101', '000110100011')

        while CI < 7:
            binCI = numberConversor(CI, "dec->bin", 9)
            parseBin = str(cacheController(binCI[1::]))

            CI += 1

            # FUNÇÃO DE LOAD
            if parseBin[0:4] == "0001":
                print("Instrução 0001 - LDA:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"RI: {parseBin}")
                print(f"Reg: {REG}")

            # FUNÇÃO DE STORE
            if parseBin[0:4] == "0010":
                REG = cacheController(parseBin[4::])
                print("Instrução 0010 - STR:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {REG}")
                print(f"Reg0: {REG}")
                print(f"RI: {parseBin}")

            # FUNÇÃO DE ADIÇÃO
            if parseBin[0:4] == "0011":
                result = int(numberConversor(REG, 'bin->dec', 8)) + \
                    int(numberConversor(cacheController(
                        parseBin[4::]), 'bin->dec', 8))
                REG = numberConversor(result, 'dec->bin', 9)
                REG = REG[1::]

                print("Instrução 0011 - ADD:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"Reg0: {REG}")
                print(f"RI: {parseBin}")

            # FUNÇÃO DE SUBTRAÇÃO
            if parseBin[0:4] == "0100":
                result = int(numberConversor(REG, 'bin->dec', 8)) - \
                    int(numberConversor(cacheController(
                        parseBin[4::]), 'bin->dec', 8))
                REG = numberConversor(result, 'dec->bin', 9)
                REG = REG[1::]

                print("Instrução 0100 - SUB:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"Reg0: {REG}")
                print(f"RI: {parseBin}")

            # FUNÇÃO JZ
            if parseBin[0:4] == "0101":
                tmpReg = numberConversor(
                    cacheController(parseBin[4::]), 'bin->dec', 8)
                if tmpReg == 0:
                    CI = tmpReg

                print("Instrução 0101 - JZ:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"Reg0: {REG}")
                print(f"CI: {numberConversor(CI, 'dec->bin', 9)}")

            # FUNÇÃO JP
            if parseBin[0:4] == "0110":
                tmpReg = numberConversor(
                    cacheController(parseBin[4::]), 'bin->dec', 8)
                if tmpReg > 0:
                    CI = tmpReg

                print("Instrução 0110 - JP:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"Reg0: {REG}")
                print(f"CI: {numberConversor(CI, 'dec->bin', 9)}")

            # FUNÇÃO JM
            if parseBin[0:4] == "0111":
                tmpReg = numberConversor(
                    cacheController(parseBin[4::]), 'bin->dec', 8)
                if tmpReg < 0:
                    CI = tmpReg

                print("Instrução 0111 - JN:")
                print(f"REM: {parseBin[4::]}")
                print(f"RDM: {cacheController(parseBin[4::])}")
                print(f"Reg0: {REG}")
                print(f"CI: {numberConversor(CI, 'dec->bin', 9)}")

            # FUNÇÃO JUMP
            if parseBin[0:4] == "1000":
                tmpReg = numberConversor(
                    cacheController(parseBin[4::]), 'bin->dec', 8)

                print("Instrução 1000 - JUMP:")
                if tmpReg != 0:
                    CI = tmpReg
                    print(f"REM: {parseBin[4::]}")
                    print(f"RDM: {cacheController(parseBin[4::])}")
                    print(f"Reg0: {REG}")
                    print(f"CI: {numberConversor(CI, 'dec->bin', 9)}")
                else:
                    print(
                        "Função JUMP não executada por falta de informação no operador")

            # FIM DO PROGRAMA
            if parseBin[0:4] == "0000":
                print("Instrução 0000 - HLT:")
                print("Fim da execução")
    if cmd == 2:
        # visualiza o estado atual da MP
        for i in range(0, len(MP)):
            print(''.join(str(e) for e in MP[i]))
        print()
    if cmd == 3:
        # visualiza o estado atual do cache
        for i in range(0, len(CACHE)):
            print(''.join(str(e) for e in CACHE[i]))
        print()
    if cmd == 4:
        runner = False


    Selecione uma opção:
    1 - Carregar programa
    2 - Visualizar MP
    3 - Visualizar cache
    4 - Encerrar
    
1
Success: o dado foi armazenado no index 0 da memória.
Success: o dado foi armazenado no index 1 da memória.
Success: o dado foi armazenado no index 2 da memória.
Success: o dado foi armazenado no index 3 da memória.
Success: o dado foi armazenado no index 4 da memória.
Success: o dado foi armazenado no index 5 da memória.
Success: o dado foi armazenado no index 6 da memória.
Success: o dado foi armazenado no index 180 da memória.
Success: o dado foi armazenado no index 181 da memória.
Instrução 0001 - LDA:
REM: 10110100
RDM: 001010110011
RI: 000110110100
Reg: 
Instrução 0100 - SUB:
REM: 10110101
RDM: 000110100011
Reg0: 00000000
RI: 010010110101
Instrução 0111 - JN:
REM: 00000101
RDM: 101010110101
Reg0: 00000000
CI: 000000011
Instrução 1000 - JUMP:
Função JUMP não executada por falta de informação no operador
Instrução 0000 - HLT:
Fim da execução

    Selecione uma 